In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
path = '/Volumes/eSSD0/Papers/JoC_SOLu/Data/drivers/'

case = 'hiECS.3'

sst = xr.load_dataset(path+case+'/sst_cpl.nc')
sst_filled = sst.fillna(-1.8) #fill masked values with -1.8 since that's what CAM considers landmass
ice = xr.load_dataset(path+case+'/ice_cov.nc')
ice_filled = ice.fillna(1) #fill masked values with 1

In [3]:
sst #what did i do to make these files again? check history

<xarray.Dataset>
Dimensions:     (time: 12, d2: 2, nlat: 116, nlon: 100)
Coordinates:
  * time        (time) object 0100-02-01 00:00:00 ... 0101-01-01 00:00:00
    TLONG       (nlat, nlon) float64 323.3 326.9 330.5 ... 317.8 319.3 320.8
    TLAT        (nlat, nlon) float64 -79.5 -79.5 -79.5 ... 68.36 68.26 68.21
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    time_bound  (time, d2) object 0100-01-01 00:00:00 ... 0101-01-01 00:00:00
    SST_cpl     (time, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
Attributes: (12/13)
    title:           b.e12.pi-control-driver.T31_g37.hiECS.3
    history:         Thu Jan  5 11:08:09 2023: ncrename -v SST,SST_cpl temp.n...
    Conventions:     CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-curr...
    contents:        Diagnostic and Prognostic Variables
    source:          CCSM POP2, the CCSM Ocean Component
    revision:        $Id: tavg.F90 58852 2014-04-04 21:28:27Z mlevy@ucar.edu $
    ...              ...
    start_time:      This dataset was created on 2022-12-05 at 17:48:20.1
    cell_methods:    cell_methods = time: mean ==> the variable values are av...
    nsteps_total:    19370
    tavg_sum:        2678400.0
    tavg_sum_qflux:  2678400.0
    NCO:             netCDF Operators version 4.9.1 (Homepage = http://nco.sf...

In [31]:
reference = xr.load_dataset(path+'sst_HadOIBl_bc_1.9x2.5_clim_pi_c101028.nc') #read in historical SST
time = reference['time'][:12].values #rip attributes
date = reference['date'][:12].values
date_attrs = reference['date'].attrs
datesec = reference['datesec'][:12].values
datesec_attrs = reference['datesec'].attrs
lon = sst_filled['TLONG'].values
lat = sst_filled['TLAT'].values #rip lat/lon arrays from the sst/ice datasets.

In [32]:
reference #checking date/datesec variables as a reference for setting vars in env_run.xml

<xarray.Dataset>
Dimensions:            (time: 12, lon: 144, lat: 96)
Coordinates:
  * lon                (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lat                (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * time               (time) object 0000-01-16 12:00:00 ... 0000-12-16 12:00:00
Data variables:
    date               (time) int32 116 215 316 416 516 ... 916 1016 1116 1216
    datesec            (time) int32 43200 0 43200 0 43200 ... 0 43200 0 43200
    ice_cov            (time, lat, lon) float32 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    ice_cov_prediddle  (time, lat, lon) float32 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    SST_cpl            (time, lat, lon) float32 -1.8 -1.8 -1.8 ... -1.8 -1.8
    SST_cpl_prediddle  (time, lat, lon) float32 -1.8 -1.8 -1.8 ... -1.8 -1.8
Attributes:
    history:  2010-10-28 14:24:16 regrid -i /datalocal/nrcm-cgd/jcaron/1850ss...

In [33]:
sst_ready = sst_filled['SST_cpl'].values #just want the array, not the metadata
sst_attrs = reference['SST_cpl'].attrs #just want the metadata, not the array

ice_ready = ice_filled['ice_cov'].values
ice_attrs = reference['ice_cov'].attrs

In [34]:
#populate a nested dictionary using the cleaned data
nested = {
    "dims": {
        'time': 12,
        'nlat': 116,
        'nlon': 100
    },
    "coords": {
        "time": {
            "dims": "time",
            "data": time
            },
        "lat": {
            "dims": ["nlat","nlon"],
            "data": lat
            },
        "lon": {
            "dims": ["nlat","nlon"],
            "data":lon
            }
        },
    "data_vars": {
        "date": {
            "dims": "time",
            "data": date,
            "attrs": date_attrs
        },
        "datesec": {
            "dims": "time",
            "data": datesec,
            "attrs": datesec_attrs
        },
        "SST_cpl":
            {
            "dims":[ "time","nlat","nlon"],
            "data": sst_ready,
            "attrs": sst_attrs
        },
        "ice_cov":
            {
            "dims": ["time","nlat","nlon"],
            "data": ice_ready,
            "attrs": ice_attrs
        }
    }
}

#define encoding params, important to specify that there is NO fill value, CESM doesn't like that
encoding = {'lat': {'zlib': False, '_FillValue': None},
            'lon': {'zlib': False, '_FillValue': None},
            'ice_cov': {'_FillValue': None,
                  'chunksizes': (1, 8, 10),
                  'complevel': 1,
                  'zlib': True},
            'SST_cpl': {'_FillValue': None,
                  'chunksizes': (1, 8, 10),
                  'complevel': 1,
                  'zlib': True}
            }

In [35]:
dataset = xr.Dataset.from_dict(nested) #create dataset from nested dict
file = 'b.e12.pi-control-driver.T31_g37.'+case+'.sstice.nc' 
dataset.to_netcdf(file, encoding=encoding) #creat netcdf from dataset